# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f"https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()[0]
data

{'avgTotalVolume': 84593632,
 'calculationPrice': 'tops',
 'change': 0.44,
 'changePercent': 0.0033,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': '15 minute delayed price',
 'highTime': 1673544577349,
 'iexAskPrice': 133.92,
 'iexAskSize': 100,
 'iexBidPrice': 131.32,
 'iexBidSize': 102,
 'iexClose': 133.93,
 'iexCloseTime': 1673545474469,
 'iexLastUpdated': 1673545474469,
 'iexMarketPercent': 0.020069266884063998,
 'iexOpen': 134.11,
 'iexOpenTime': 1673533801464,
 'iexRealtimePrice': 133.93,
 'iexRealtimeSize': 14,
 'iexVolume': 768919,
 'lastTradeTime': 1673545474469,
 'latestPrice': 133.93,
 'latestSource': 'IEX real time price',
 'latestTime': '12:44:34 PM',
 'latestUpdate': 1673545474469,
 'latestVolume': None,
 'low': None,
 '

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
marketCap = data['marketCap']
print(marketCap)

2130574243740


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
myColumns = ['Ticker', 'Stock Price', 'Market Cap', 'Number of shares to buy']
finalDataFrame = pd.DataFrame(columns = myColumns)
finalDataFrame

,Ticker,Stock Price,Market Cap,Number of shares to buy


In [7]:
finalDataFrame.append(
pd.Series([
    symbol,
    price,
    marketCap,
    'N/A'
], index = myColumns),
    ignore_index = True
)

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,AAPL,133.93,2130574243740,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
finalDataFrame = pd.DataFrame(columns = myColumns)
for stock in stocks['Ticker'][:5]:
    api_url = f"https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()[0]
    finalDataFrame = finalDataFrame.append(pd.Series([
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = myColumns
    ), ignore_index = True)   

In [9]:
finalDataFrame

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,157.44,46613581978,N/A
1,AAL,16.49,10716865627,N/A
2,AAP,152.97,9196287020,N/A
3,AAPL,133.93,2130574243740,N/A
4,ABBV,152.91,270418354478,N/A


# Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i+n]

In [18]:
symbolGroups = list(chunks(stocks['Ticker'], 100))
symbolStrings = []
for i in range(0, len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
for symbolString in symbolStrings:
    batchApiUrl = f"https://api.iex.cloud/v1/data/core/quote/{symbolString}?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batchApiUrl).json()[0]
    for symbol in symbolString.split(','):
        finalDataFrame = finalDataFrame.append(
            pd.Series([
                symbol,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ], index = myColumns),
            ignore_index = True
        )
finalDataFrame[]

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,157.44,46613581978,N/A
1,AAL,16.49,10716865627,N/A
2,AAP,152.97,9196287020,N/A
3,AAPL,133.93,2130574243740,N/A
4,ABBV,152.91,270418354478,N/A
...,...,...,...,...
605,YUM,129.18,36388438530,N/A
606,ZBH,129.18,36388438530,N/A
607,ZBRA,129.18,36388438530,N/A
608,ZION,129.18,36388438530,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [28]:
def portfolioSize():
    size = input("Enter the value of your portfolio: ")
    try:
        val = float(size)
        return size
    except ValueError:
        print("That's not a number! \nPlease enter a number.")
        portfolioSize()
value = int(portfolioSize())
type(value)

Enter the value of your portfolio: 10000000


int

In [54]:
positionSize = value/len(finalDataFrame.index)
for i in range(0, len(finalDataFrame.index)):
    finalDataFrame.loc[i, 'Number of shares to buy'] = math.floor(positionSize/finalDataFrame.loc[i, 'Stock Price'])
finalDataFrame[:20]

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,157.440,46613581978,104
1,AAL,16.490,10716865627,994
2,AAP,152.970,9196287020,107
3,AAPL,133.930,2130574243740,122
4,ABBV,152.910,270418354478,107
5,A,157.775,46712766111,103
6,AAL,157.775,46712766111,103
7,AAP,157.775,46712766111,103
8,AAPL,157.775,46712766111,103
9,ABBV,157.775,46712766111,103


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [50]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
finalDataFrame.to_excel(writer, 'Recommended Trades', index = False, encoding='UTF-8')

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [51]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

integer_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [52]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [53]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', integer_format],
    'D': ['Number of shares to buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [55]:
writer.save()